In [1]:
import os
os.chdir('./../')

In [2]:
from dotenv import load_dotenv

from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dataclasses import dataclass

load_dotenv(override=True)

from common.constants import Constants

In [3]:
@dataclass
class Message:
    content: str

In [4]:
class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("Simple")

    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        return Message(content=f"This is {self.id.type}-{self.id.key}. You said '{message.content}' and I disagree.")

In [5]:
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())

AgentType(type='simple_agent')

In [6]:
runtime.start()

In [7]:
agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("Well hi there!"), agent_id)
print(">>>", response.content)

>>> This is simple_agent-default. You said 'Well hi there!' and I disagree.


In [8]:
await runtime.stop()
await runtime.close()

In [9]:
from autogen_core.models import ModelFamily

class MyLLMAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        model_client = OpenAIChatCompletionClient(model=Constants.GEMINI_MODEL_LITE,
                                                  api_key=os.getenv(Constants.GOOGLE_API_KEY),
                                                  model_info={'vision': True,
                                                              'function_calling': True,
                                                              'json_output': True,
                                                              'family': ModelFamily.GEMINI_2_0_FLASH,
                                                              'structured_output': True})
        
        self._delegate = AssistantAgent("LLMAgent", model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        print(f"{self.id.type} received message: {message.content}")
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        reply = response.chat_message.content
        print(f"{self.id.type} responded: {reply}")
        return Message(content=reply)

In [10]:
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await MyLLMAgent.register(runtime, "LLMAgent", lambda: MyLLMAgent())

AgentType(type='LLMAgent')

In [11]:
runtime.start()  # Start processing messages in the background.
response = await runtime.send_message(Message("Hi there!"), AgentId("LLMAgent", "default"))
print(">>>", response.content)
response =  await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(">>>", response.content)
response = await runtime.send_message(Message(response.content), AgentId("LLMAgent", "default"))

LLMAgent received message: Hi there!
LLMAgent responded: Hi there! How can I help you today?
>>> Hi there! How can I help you today?
>>> This is simple_agent-default. You said 'Hi there! How can I help you today?' and I disagree.
LLMAgent received message: This is simple_agent-default. You said 'Hi there! How can I help you today?' and I disagree.
LLMAgent responded: I apologize if my previous response was not to your liking. As a large language model, I am still under development and always striving to improve. Can you please tell me what you would prefer me to say or how I can assist you better? I want to make sure I'm being as helpful as possible.


In [12]:
await runtime.stop()
await runtime.close()

In [13]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model=Constants.GEMINI_MODEL_LITE,
                                                  api_key=os.getenv(Constants.GOOGLE_API_KEY),
                                                  model_info={'vision': True,
                                                              'function_calling': True,
                                                              'json_output': True,
                                                              'family': ModelFamily.GEMINI_2_0_FLASH,
                                                              'structured_output': True},
                                                  temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model=Constants.GEMINI_MODEL_LITE,
                                                  api_key=os.getenv(Constants.GOOGLE_API_KEY),
                                                  model_info={'vision': True,
                                                              'function_calling': True,
                                                              'json_output': True,
                                                              'family': ModelFamily.GEMINI_2_0_FLASH,
                                                              'structured_output': True},
                                                  temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [14]:
JUDGE = "You are judging a game of rock, paper, scissors. The players have made these choices:\n"

class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model=Constants.GEMINI_MODEL_LITE,
                                                  api_key=os.getenv(Constants.GOOGLE_API_KEY),
                                                  model_info={'vision': True,
                                                              'function_calling': True,
                                                              'json_output': True,
                                                              'family': ModelFamily.GEMINI_2_0_FLASH,
                                                              'structured_output': True},
                                                  temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        instruction = "You are playing rock, paper, scissors. Respond only with the one word, one of the following: rock, paper, or scissors."
        message = Message(content=instruction)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message, inner_1)
        response2 = await self.send_message(message, inner_2)
        result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\n"
        judgement = f"{JUDGE}{result}Who wins?"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + response.chat_message.content)

In [15]:
runtime = SingleThreadedAgentRuntime()
await Player1Agent.register(runtime, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(runtime, "player2", lambda: Player2Agent("player2"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))
runtime.start()

In [16]:
agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
print(response.content)

Player 1: rock
TERMINATE
Player 2: paper
TERMINATE
Paper covers rock, so Player 2 wins. TERMINATE


In [17]:
await runtime.stop()
await runtime.close()